In [ ]:
!pip install ta

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas_datareader as pdr
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import TimeseriesGenerator
import os
import gc
import traceback
import datatable as dt
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd, numpy as np
from tensorflow.keras import layers
import tensorflow_probability as tfp
import tensorflow.keras.backend as K
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
import keras

In [ ]:
# Set graph style and font
sns.set()
# Change the axes' title and label size to 18 & 16 by default and default figure size, and make title bold
# Axes formatter limit will only display scientific notation if it's > 10^7 (or 10 million JPY) or < 10^-5
plt.rcParams.update({'axes.titleweight': 'bold','figure.figsize': (16,10),'axes.titlesize': 18,'axes.labelsize': 16, 
                     'legend.fontsize': 12, 'xtick.labelsize': 12, 'ytick.labelsize': 12, 'font.family': 'serif', 
                     'axes.formatter.limits':'-5, 7'}) 

# Loading data

In [ ]:
# For Kaggle use only
directory_path = "/kaggle/input/392-crypto-currency-pairs-at-minute-resolution/"
BTC = pd.read_csv(directory_path + 'btcusd.csv')
print(BTC.head())
ETH = pd.read_csv(directory_path + 'ethusd.csv')
print(ETH.head())
LTC = pd.read_csv(directory_path + 'ltcusd.csv')
print(LTC.head())

In [ ]:
# Convert to human timestamp
BTC['time'] = pd.to_datetime(BTC['time'], unit='ms')
ETH['time'] = pd.to_datetime(ETH['time'], unit='ms')
LTC['time'] = pd.to_datetime(LTC['time'], unit='ms')

In [ ]:
BTC.describe(include='all')

In [ ]:
ETH.describe(include='all')

In [ ]:
LTC.describe(include='all')

In [ ]:
# Make copies of these df before doing further
BTC_copy = BTC.copy()
ETH_copy = ETH.copy()
LTC_copy = LTC.copy()

In [ ]:
# Set time as index for plotting
BTC.set_index('time', inplace=True)
ETH.set_index('time', inplace=True)
LTC.set_index('time', inplace=True)
BTC

# EDA

In [ ]:
plt.plot(BTC.index, BTC.close)
plt.xlabel('Date')
plt.ylabel('Price in USD')
plt.title('Price of BTC over years')
plt.show()

In [ ]:
plt.plot(ETH.index, ETH.close)
plt.xlabel('Date')
plt.ylabel('Price in USD')
plt.title('Price of ETH over years')
plt.show()

In [ ]:
plt.plot(LTC.index, LTC.close)
plt.xlabel('Date')
plt.ylabel('Price in USD')
plt.title('Price of LTC over years')
plt.show()

In [ ]:
print(BTC.isnull().sum())
print(ETH.isnull().sum())
print(LTC.isnull().sum())

No null values at all, no need to drop any data

In [ ]:
# Use only data from the last 2 years for modelling
BTC_2yr = BTC['2020-01-01':]
ETH_2yr = ETH['2020-01-01':]
LTC_2yr = LTC['2020-01-01':]
BTC_2yr

# Feature engineering

In [ ]:
def upper_shadow(df): return df['high'] - np.maximum(df['close'], df['open'])
def lower_shadow(df): return np.minimum(df['close'], df['open']) - df['low']

def get_features(df, row = False):
    df_feat = df
    df_feat['spread'] = df_feat['high'] - df_feat['low']
    df_feat['upper_shadow'] = upper_shadow(df_feat)
    df_feat['lower_shadow'] = lower_shadow(df_feat)
    df_feat['close-open'] = df_feat['close'] - df_feat['open']
    df_feat['SMA_7'] = df_feat.iloc[:,1].rolling(window=7).mean()
    df_feat['SMA_14'] = df_feat.iloc[:,1].rolling(window=14).mean()
    df_feat['SMA_21'] = df_feat.iloc[:,1].rolling(window=21).mean()
    # Create the STD_DEV feature for the past 7 days
    df_feat['STD_DEV_7'] = df_feat.iloc[:,1].rolling(window=7).std()
    # Drop the NA rows created by the SMA indicators
    df_feat.dropna(inplace = True)
    return df_feat

In [ ]:
BTC_2yr = get_features(BTC_2yr)
BTC_2yr

In [ ]:
plt.plot(BTC_2yr.index, BTC_2yr['close'])
plt.show()

In [ ]:
BTC_y = BTC_2yr['close']
BTC_X = BTC_2yr.drop('close', axis=1)
BTC_X

In [ ]:
BTC_y_hr = BTC_y.resample('h').mean()
BTC_X_hr = BTC_X.resample('h').mean()
print(BTC_X_hr.shape, BTC_y_hr.shape)

# For testing

In [ ]:
# # Only URL works, S3URI requires credentials
# BTC_USD_S3URI = "s3://cryptocurrency-forecasting/raw_data/btcusd.csv"
# BTC_USD_URL = "https://cryptocurrency-forecasting.s3.ap-northeast-1.amazonaws.com/raw_data/btcusd.csv"
# ETH_USD_S3URI = "s3://cryptocurrency-forecasting/raw_data/etcusd.csv"
# ETH_USD_URL = "https://cryptocurrency-forecasting.s3.ap-northeast-1.amazonaws.com/raw_data/etcusd.csv"
# LTC_USD_S3URI = "s3://cryptocurrency-forecasting/raw_data/ltcusd.csv"
# LTC_USD_URL = "https://cryptocurrency-forecasting.s3.ap-northeast-1.amazonaws.com/raw_data/ltcusd.csv"

# def get_data(crypto, last_rows=500000):
#     """ Get dataset of selected crypto from cloud storage
#     500,000 data points (mins) is roughly equivalent to 1 year worth of data
#     # Params: chosen crypto by the user from the front end """
#     if crypto == "BTC":
#         df_full = pd.read_csv(BTC_USD_URL)
#         n_rows = len(df_full)
#         df = pd.read_csv(BTC_USD_URL, skiprows=range(1, n_rows - last_rows))
#     elif crypto == 'ETH':
#         df_full = pd.read_csv(ETH_USD_URL)
#         n_rows = len(df_full)
#         df = pd.read_csv(ETH_USD_URL, skiprows=range(1, n_rows - last_rows))
#     elif crypto == 'LTC':
#         df_full = pd.read_csv(LTC_USD_URL)
#         n_rows = len(df_full)
#         df = pd.read_csv(LTC_USD_URL, skiprows=range(1, n_rows - last_rows))
#     return df

# def MinMax_fit_transform(X):
#     MinMaxscaler = MinMaxScaler()
#     MinMaxscaler.fit(X)
#     X_transformed = MinMaxscaler.fit_transform(X)
#     return X_transformed, MinMaxscaler

# def MinMax_reverse_fit_transform(MinMaxscaler, X_transformed):
#     X = MinMaxscaler.inverse_transform(X_transformed)
#     return X


# # For testing
# LTC = get_data('LTC')
# print(LTC)
# LTC_transformed, MinMaxscaler = MinMax_fit_transform(LTC)
# print(LTC_transformed)
# LTC_original = MinMax_reverse_fit_transform(MinMaxscaler, LTC_transformed)
# print(LTC_original)

# Modelling

## Baseline model: linear regression

In [ ]:
# 70% for training, 30% for testing
index_70pct = int(len(BTC_X)*0.7)
BTC_X_train = BTC_X[:index_70pct]
BTC_X_test = BTC_X[index_70pct:]
BTC_y_train = BTC_y[:index_70pct]
BTC_y_test = BTC_y[index_70pct:]
print(BTC_X_train.shape)
print(BTC_y_test.shape)

In [ ]:
linreg = LinearRegression()
linreg.fit(BTC_X_train.shift(-1000), BTC_y_train)

In [ ]:
BTC_y_pred = linreg.predict(BTC_X_test)
BTC_y_pred


BTC_X_test = BTC_X_test.shift()

In [ ]:
len(BTC_y_test)

In [ ]:
mse = mean_squared_error(BTC_y_pred, BTC_y_test)
mse

In [ ]:
# plt.plot(BTC_y_train.index, BTC_y_train, color = 'y', label ='Training prices')
# plt.plot(BTC_y_test.index, BTC_y_test, color = 'y', label ='Test prices')
plt.plot(BTC_y_test.index, BTC_y_pred, color = 'b', label ='Predicted prices')
plt.legend()
plt.show()

In [ ]:
print(linreg.coef_)
print(linreg.intercept_)

# LSTM

In [ ]:
# Use simple 70:30 train minutes data
print(BTC_X_train.shape, BTC_X_test.shape, BTC_y_train.shape, BTC_y_test.shape)

In [ ]:
seq_len = 10000
window_size = seq_len - 1
batch_size = 128

In [ ]:
# print(BTC_X)
print(BTC_y)

In [ ]:
# Rescaling all features
values_X = BTC_X_hr.values
values_y = BTC_y_hr.values.reshape(-1, 1)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_X = scaler.fit_transform(values_X)
# scaled_y = scaler.fit_transform(values_y_train)
print(scaled_X_train.shape, scaled_y_train.shape, scaled_X_test.shape, scaled_y_test.shape)

In [ ]:
# Memory saving function 
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype.name
        
        if col_type not in ['object', 'category', 'datetime64[ns, UTC]', 'datetime64[ns]']:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [ ]:
# Reduce memory usage of BTC_X
BTC_X = reduce_mem_usage(BTC_X)
# BTC_y = reduce_mem_usage(BTC_y)  # This results in an error

In [ ]:
# seq_x = to_sequences(BTC_X, )
BTC_X

In [ ]:
print(BTC_X_hr.shape, BTC_y_hr.shape)

In [ ]:
last_1000_hr_X = BTC_X_hr.tail(1000)
# last_1000_hr_X
last_1000_X_np = last_1000_hr_X.values
last_1000_X_np = np.expand_dims(last_1000_X_np, 0)
print(last_1000_X_np.shape)

In [ ]:
def unison_shuffled_copies(X, y):
    assert len(X) == len(y)
    p = np.random.permutation(len(X))
    return X[p], y[p]

# Convert data to sequences
def to_sequences(data_X, data_y, seq_len):
    d_X = []
    d_y = []

    for index in range(len(data_X) - seq_len - 24): # Exclude last 1024 values
        d_X.append(data_X[index: index + seq_len])
        d_y.append(data_y[index + seq_len + 24])  # 1 value 24 hours into the future

    return np.array(d_X), np.array(d_y)

def preprocess(X, y, seq_len, train_split):

    data_X = to_sequences(X, seq_len)
    data_y = to_sequences(y, seq_len)

    num_train = int(train_split * data_X.shape[0])

    X_train = data_X[:num_train, :-1, 11]  # Shape is (number of observations, , number of features)
    y_train = data_y[:num_train, -1, 1]

    X_test = data_X[num_train:, :-1, 11]
    y_test = data_y[num_train:, -1, 1]

    return X_train, y_train, X_test, y_test


# X_train, y_train, X_test, y_test = preprocess(BTC_X_hr, BTC_y_hr, seq_len, train_split = 0.7)

# print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
np.isnan(scaled_X_train).sum()
np.isnan(scaled_X_test).sum()

In [ ]:
BTC_X_seq, BTC_y_seq = to_sequences(scaled_X, values_y, seq_len)

BTC_X_seq_train = BTC_X_seq[:-1440]
BTC_X_seq_test = BTC_X_seq[-1440:]
BTC_y_seq_train = BTC_y_seq[:-1440]
BTC_y_seq_test = BTC_y_seq[-1440:]

BTC_X_seq_train, BTC_y_seq_train = unison_shuffled_copies(BTC_X_seq_train, BTC_y_seq_train)

print(BTC_X_seq_train.shape, BTC_X_seq_test.shape, BTC_y_seq_train.shape)

In [ ]:
# Old version without minMaxscaler
# Fill in nan with linear interpolation
BTC_X_hr = BTC_X_hr.interpolate()
BTC_y_hr = BTC_y_hr.interpolate()

BTC_X_seq, BTC_y_seq = to_sequences(BTC_X_hr, BTC_y_hr, seq_len)
BTC_y_seq = np.expand_dims(BTC_y_seq, -1)

BTC_X_seq_train = BTC_X_seq[:-1440]
BTC_X_seq_test = BTC_X_seq[-1440:]
BTC_y_seq_train = BTC_y_seq[:-1440]
BTC_y_seq_test = BTC_y_seq[-1440:]

BTC_X_seq_train, BTC_y_seq_train = unison_shuffled_copies(BTC_X_seq_train, BTC_y_seq_train)

print(BTC_X_seq_train.shape, BTC_X_seq_test.shape, BTC_y_seq_train.shape)

In [ ]:
print(np.isnan(BTC_X_seq_train).sum())
print(np.isnan(BTC_y_seq_train).sum())

In [ ]:
# norm = tf.keras.layers.Normalization(axis=[-2, -1]) # normalize across seq & features
# norm.adapt(BTC_X_seq)
def init_model():
    model = Sequential()
#     model.add(norm)
    model.add(LSTM(units=128, return_sequences=True, activation='tanh', input_shape=(BTC_X_seq_train.shape[1], \
                                                                                    BTC_X_seq_train.shape[2])))
    model.add(LSTM(units=64, activation='tanh'))
    model.add(Dense(64, activation = 'LeakyReLU'))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='linear'))
    return model

def compile_model(model): 
#     Adam_opt = keras.optimizers.adam_v2(0.0005, beta_1=0.9, beta_2=0.999, amsgrad=False)
    model.compile(loss='mse', 
        optimizer='rmsprop',
        metrics='mae')
    return model

model = init_model()
compile_model(model)
model.summary()

In [ ]:
es = tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss')
history = model.fit(BTC_X_seq_train, BTC_y_seq_train, epochs=15, batch_size=batch_size,
                    validation_split=0.3, verbose=1, shuffle=False, callbacks=[es])
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
BTC_y_pred_train = model.predict(BTC_X_seq_train)
print(BTC_y_pred_train)

BTC_y_pred_test = model.predict(BTC_X_seq_test)
print(BTC_y_pred_test)

# plt.plot(BTC_y_seq_train, color = 'y', label='Actual train prices')
plt.plot(BTC_y_seq_test, color = 'g', label = 'Actual test prices')
# plt.plot(BTC_y_pred_train, color = 'r', label = 'Predicted train prices')
plt.plot(BTC_y_pred_test, color = 'b', label = 'Predicted test prices')
plt.legend()
plt.show()

In [ ]:
# latest_24_hours = BTC_y_pred[]

# Alternate LSTM